Extracting polygons data from kml files

In [ ]:
from pykml import parser
from os import path
import pandas as pd
from bs4 import BeautifulSoup

#Extracting data from .kml file downloaded from https://registry.verra.org/app/search/VCS
with open('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.kml', 'r') as kml_file:
    kml_content = parser.parse(kml_file).getroot().Document


names = []
descriptions = []
coordinates = []


def parse_coordinates(coordinate_string):
    coordinate_list = coordinate_string.strip().split()
    return [(float(coordinate.split(',')[1]), float(coordinate.split(',')[0])) for coordinate in coordinate_list]

def parse_description(description):
    if description is None:
        return None
    soup = BeautifulSoup(description, 'html.parser')
    table = soup.find('table')
    data = {}
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) > 1:
            key = cells[0].text.strip().replace(':', '')
            value = cells[1].text.strip()
            data[key] = value
    return data


for folder in kml_content.Folder:

    for placemark in folder.Placemark:
        name = placemark.name.text if hasattr(placemark, 'name') else None
        description = parse_description(placemark.description.text) if hasattr(placemark, 'description') and placemark.description.text is not None else None
        if hasattr(placemark, 'Point'):
            coordinate = parse_coordinates(placemark.Point.coordinates.text)
        elif hasattr(placemark, 'LineString'):
            coordinate = parse_coordinates(placemark.LineString.coordinates.text)
        else:
            coordinate = None
        names.append(name)
        descriptions.append(description)
        coordinates.append(coordinate)


df = pd.DataFrame({
    'Name': names,
    'Description': descriptions,
    'Coordinates': coordinates
})


df.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/kml_files/03-Dingxi_Project boundary.csv', index=False)

##  Concatinating Polygons with trees and area 

In [ ]:
import pandas as pd
import glob


path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_With_tree_number'  
all_files = glob.glob(path + "/*.csv")


dfs = []


for filename in all_files:

    df = pd.read_csv(filename, index_col=None, header=0)
    
    dfs.append(df)


concatenated_df_TIST = pd.concat(dfs, axis=0, ignore_index=True)


concatenated_df_TIST.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_With_tree_number/concatenated_file.csv', index=False)

In [ ]:
import pandas as pd


file_path = 'TIST_Projects/KML_737_02NOV2011.csv'


concatenated_df = pd.read_csv(file_path)


concatenated_df.info()


In [ ]:
# CLEANING_TIST_Data _clening

df = concatenated_df


cleaned_data = []


for i in range(0, len(df) - 1, 2):
  
    name = df.iloc[i]['Name']
    description = df.iloc[i]['Description']
    
 
    coordinates = df.iloc[i + 1]['Coordinates']
    
    cleaned_data.append([name, description, coordinates])


cleaned_df = pd.DataFrame(cleaned_data, columns=['Name', 'Description', 'Coordinates'])




cleaned_df.info()

cleaned_df.to_csv("KML_737_02NOV2011.csv", index=False)

In [ ]:
import pandas as pd


file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_With_tree_number/concatenated_file.csv'


concatenated_df = pd.read_csv(file_path)


concatenated_df.info()

In [ ]:
concatenated_df.drop(columns=['Unnamed: 3'], inplace=True)
concatenated_df.info()

In [ ]:
import pandas as pd


def expand_description(description_series):
    
    expanded_data = []
    
  
    for item in description_series:
      
        if isinstance(item, str):
            item = literal_eval(item)
        expanded_data.append(item)
    
  
    return pd.DataFrame(expanded_data)


expanded_description_df = expand_description(concatenated_df['Description'])


TIST_Cleaned_df = pd.concat([concatenated_df.drop('Description', axis=1).reset_index(drop=True), expanded_description_df], axis=1)
TIST_Cleaned_df.head()

In [ ]:
TIST_Cleaned_df.info()

In [ ]:

TIST_Cleaned_df['Trees'] = pd.to_numeric(TIST_Cleaned_df['Trees'], errors='coerce').fillna(0).astype(int)


trees_sum = TIST_Cleaned_df['Trees'].sum()


print(trees_sum)

In [ ]:
import numpy as np
import pandas as pd

TIST_Cleaned_df['Grove Area'] = TIST_Cleaned_df['Grove Area'].str.replace(' Ha', '').str.replace(',', '').replace('', np.nan)


TIST_Cleaned_df['Grove Area'] = pd.to_numeric(TIST_Cleaned_df['Grove Area'], errors='coerce')


In [ ]:

total_area = TIST_Cleaned_df['Grove Area'].sum()


print(total_area)

In [ ]:
TIST_Cleaned_df.info()

In [ ]:
TIST_Cleaned_df['Coordinates']

In [ ]:
import pandas as pd
import simplekml

def row_to_kml(row, index, folder_path):
    kml = simplekml.Kml()
    polygon = kml.newpolygon(name=f"Polygon{index}")

    coordinates = eval(row['Coordinates'])
    polygon.outerboundaryis = coordinates
    kml.save(f"{folder_path}/Polygon{index}.kml")

#
df = TIST_Cleaned_df

folder_path = "kml_files/Individual_Polygons_kml"
for index, row in df.iterrows():
    row_to_kml(row, index, folder_path)

In [ ]:
import pandas as pd
import simplekml
import ast   

def row_to_kml(row, index, folder_path):
    kml = simplekml.Kml()
    polygon = kml.newpolygon(name=f"Polygon{index}")
 
    coordinates_str = row['Coordinates']
    coordinates_list = ast.literal_eval(coordinates_str)
    
    
    formatted_coordinates = [(lon, lat, 0.0) for lon, lat in coordinates_list]
    
    polygon.outerboundaryis = formatted_coordinates
    
    kml.save(f"{folder_path}/Polygon{index}.kml")

 
df = TIST_Cleaned_df

folder_path = "kml_files/Individual_Polygons_kml"
for index, row in df.iterrows():
    row_to_kml(row, index, folder_path)

In [ ]:
import pandas as pd
import simplekml
import ast   

def row_to_kml(row, index, folder_path):
    kml = simplekml.Kml()
    polygon = kml.newpolygon(name=f"Polygon{index}")
    
 
    coordinates_str = row['Coordinates']
    coordinates_list = ast.literal_eval(coordinates_str)
  
    formatted_coordinates = [(lat, lon, 0.0) for lon, lat in coordinates_list]
    
    polygon.outerboundaryis = formatted_coordinates
        # Set polygon style
    polygon.style.polystyle.color = simplekml.Color.changealphaint(10, simplekml.Color.white)  
    polygon.style.linestyle.width = 5  
    polygon.style.linestyle.color = simplekml.Color.blue   
    file_name = f"Polygon{index}"
    kml.save(f"{folder_path}/Polygon{index}.kml")
    return file_name

 
df = TIST_Cleaned_df

folder_path = "kml_files/Individual_Polygons_kml"
kml_file_names = []

for index, row in df.iterrows():
    file_name = row_to_kml(row, index, folder_path)
    kml_file_names.append(file_name)
 
df['kml_file_name'] = kml_file_names

In [ ]:
df.tail()

In [ ]:
df.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_With_tree_number/TIST_Meta.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
 
df['Tract Date'] = pd.to_datetime(df['Tract Date'])

 
df['Year'] = df['Tract Date'].dt.year

 
year_counts = df['Year'].value_counts().sort_index()

 
print("Yearly Tract Counts:")
print(year_counts)
 
plt.figure(figsize=(10, 6))
year_counts.plot(kind='bar')
plt.title('Count of Tracts per Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
import pandas as pd


file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_With_tree_number/TIST_Meta.csv'
meta= pd.read_csv(file_path)


meta.info()

In [ ]:
meta.describe()

In [ ]:
import pandas as pd


file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_With_tree_number/concatenated_file.csv'


concatenated_df = pd.read_csv(file_path)


concatenated_df.info()

### Concatinating Data for polygons without Tree information

In [ ]:
import pandas as pd
import glob
 
path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_without_trees'  
all_files = glob.glob(path + "/*.csv")


dfs = []


for filename in all_files:

    df = pd.read_csv(filename, index_col=None, header=0)
    
    dfs.append(df)


concatenated_df_TIST = pd.concat(dfs, axis=0, ignore_index=True)


concatenated_df_TIST.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_without_trees/concatenated_wt.csv', index=False)

In [ ]:
file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_without_trees/concatenated_wt.csv'


concatenated_df_TIST= pd.read_csv(file_path)


concatenated_df_TIST.info()

In [ ]:
import pandas as pd
import re
from pyproj import Proj, transform

def latlong_to_utm(coords):
    """Convert lat-long coordinates to UTM."""
    
    proj_latlong = Proj(proj='latlong', datum='WGS84')
    proj_utm = Proj(proj='utm', zone=33, datum='WGS84')
     
    utm_coords = [transform(proj_latlong, proj_utm, coord[1], coord[0]) for coord in coords]
    return utm_coords

def parse_polygon_coords(polygon_str):
    """Parse the polygon coordinates from a string."""
    coords = re.findall(r"\(([^)]+)\)", polygon_str)
    return [tuple(map(float, coord.split(','))) for coord in coords]

def calculate_polygon_area(coords):
    """Calculate the area of a polygon in square meters using the Shoelace formula."""
     
    utm_coords = latlong_to_utm(coords)
    n = len(utm_coords)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += utm_coords[i][0] * utm_coords[j][1]
        area -= utm_coords[j][0] * utm_coords[i][1]
    area = abs(area) / 2.0
    return area

 
concatenated_df_TIST['Area_m2'] = concatenated_df_TIST['Polygon_Geometry'].apply(
    lambda x: calculate_polygon_area(parse_polygon_coords(x)) if isinstance(x, str) else None
)

concatenated_df_TIST.head()

In [ ]:
 
concatenated_df_TIST['Area_ha'] = concatenated_df_TIST['Polygon_Geometry'].apply(
    lambda x: calculate_polygon_area(parse_polygon_coords(x)) / 10000 if isinstance(x, str) else None
)

concatenated_df_TIST.head()

In [ ]:
total_area_ha = concatenated_df_TIST['Area_ha'].sum()
print(f"Total area covered: {total_area_ha} hectares")

In [ ]:
conca

### Komaza/Coastal Kenya Projects

In [ ]:
import pandas as pd


file_path = 'Komaza_Carbon_Project_Farms.csv'


komaza_df = pd.read_csv(file_path)


komaza_df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns  # Seaborn is used for its nice color palette
year_counts = komaza_df['PlantYear'].value_counts().sort_index()
# Generate a list of colors, one for each year
colors = sns.color_palette('hsv', len(year_counts))

# Plot with different colors for each year
year_counts.plot(kind='bar', color=colors)
plt.xlabel('Year of Plant')
plt.ylabel('Count')
plt.title('Coastal Kenya sites Count of Entries for Each Year of Plant')
plt.show()

In [ ]:
komaza_df["Coordinates"]

In [ ]:
import pandas as pd
import simplekml
import ast

# Assuming komaza_df is already loaded
output_folder = "/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files"

def format_coordinates_to_list(coordinates):
    # If coordinates are in string format, convert them to a list of tuples
    if isinstance(coordinates, str):
        coordinates = ast.literal_eval(coordinates)
    # Convert each tuple to include longitude, latitude, and altitude (0)
    return [(coord[1], coord[0], 0) for coord in coordinates]  # Note the order change to (lon, lat, alt)

# Example usage with a DataFrame
for index, row in komaza_df.iterrows():
    kml = simplekml.Kml()
    pol = kml.newpolygon(name=f"Polygon_{index}")
    # Ensure the coordinates are in the correct format
    formatted_coordinates = format_coordinates_to_list(row["Coordinates"])
    pol.outerboundaryis = formatted_coordinates  # Directly use the list of tuples
    # Save the KML file in the specified output folder
    kml.save(f"{output_folder}/Polygon_{index}.kml")

In [ ]:
import pandas as pd
import simplekml
import ast

output_folder = "/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files"

def format_coordinates_to_list(coordinates):
    if isinstance(coordinates, str):
        coordinates = ast.literal_eval(coordinates)
    return [(coord[0], coord[1], 0) for coord in coordinates]

 
kml_file_names = []

for index, row in komaza_df.iterrows():
    kml = simplekml.Kml()
    pol = kml.newpolygon(name=f"Polygon_{index}")
    formatted_coordinates = format_coordinates_to_list(row["Coordinates"])
    pol.outerboundaryis = formatted_coordinates
    pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.red)
    pol.style.linestyle.width = 5
    file_name = f"Polygon_{index}.kml"
    kml.save(f"{output_folder}/{file_name}")
    
    kml_file_names.append(file_name)
 
komaza_df['kml_file_name'] = kml_file_names

In [ ]:
komaza_df.head()

In [ ]:
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import transform
import pyproj
from functools import partial
import ast


def calculate_area_sqkm(coordinates):

    coordinates = ast.literal_eval(coordinates)
    polygon = Polygon(coordinates)
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),  
        pyproj.Proj(init='epsg:6933')  
    )
    projected_polygon = transform(project, polygon)
    area_sqm = projected_polygon.area
    area_sqkm = area_sqm / 1e6
    return area_sqkm


komaza_df['Area_sqkm'] = komaza_df['Coordinates'].apply(calculate_area_sqkm)

In [ ]:
komaza_df.head()

In [ ]:
komaza_df.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files/Komaza_preprocessed.csv', index=False)

In [ ]:



import pandas as pd


csv_file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/Komaza_preprocessed_sample - Komaza_preprocessed.csv'


df_1 = pd.read_csv(csv_file_path)

df_1.head()


In [ ]:
path="/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/NEW_LANDCOVER_SAMPLE - Sheet1.csv"
df_2=pd.read_csv(path)
df_2.head()

In [ ]:

merged_df = pd.merge(df_1, df_2, left_on='kml_file_name', right_on='label')


merged_df.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/Sample_Landcover.csv', index=False)

In [ ]:
csv_file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/NEW_LANDCOVER_SAMPLE - Sheet1.csv'


df_2 = pd.read_csv(csv_file_path)

df_2.head()

In [ ]:
import os
from fastkml import kml
from shapely.geometry import shape, Polygon
import geopandas as gpd

def kml_to_polygon(kml_path):
    try:
        with open(kml_path, 'rb') as file:  
            doc = file.read()
            k = kml.KML()
            k.from_string(doc)  
            features = list(k.features())
            placemarks = list(features[0].features())
            polygon = placemarks[0].geometry
            if not isinstance(polygon, Polygon):
                polygon = shape(polygon)
            return polygon
    except FileNotFoundError:
        print(f"File not found: {kml_path}")
        return None

geometries = []
labels = []


kml_folder_path = "/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files"

for index, row in komaza_df.iterrows():
    kml_file_name = row['kml_file_name']
    kml_path = os.path.join(kml_folder_path, kml_file_name)
    polygon = kml_to_polygon(kml_path)
    if polygon is not None:
        geometries.append(polygon)
        labels.append(row['kml_file_name'])
    else:
        print(f"Skipping missing file: {kml_path}")

gdf = gpd.GeoDataFrame({'label': labels, 'geometry': geometries})
gdf.crs = "EPSG:4326"
gdf.to_file("/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/komaza_shapefile.shp")

In [ ]:
import pandas as pd
import re
from pyproj import Proj, transform


def latlong_to_utm(coords):
    proj_latlong = Proj(proj='latlong', datum='WGS84')
    proj_utm = Proj(proj='utm', zone=33, datum='WGS84')   
    utm_coords = [transform(proj_latlong, proj_utm, coord[1], coord[0]) for coord in coords]
    return utm_coords

def parse_polygon_coords(polygon_str):
    coords = re.findall(r"\(([^)]+)\)", polygon_str)
    return [tuple(map(float, coord.split(','))) for coord in coords]

def calculate_polygon_area(coords):
    utm_coords = latlong_to_utm(coords)
    n = len(utm_coords)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += utm_coords[i][0] * utm_coords[j][1]
        area -= utm_coords[j][0] * utm_coords[i][1]
    area = abs(area) / 2.0
    return area / 10000   


komaza_df['Area_in_ha'] = komaza_df['Coordinates'].apply(
    lambda x: calculate_polygon_area(parse_polygon_coords(x)) if isinstance(x, str) else None
)

In [ ]:
import os
from fastkml import kml
from shapely.geometry import shape, Polygon
import geopandas as gpd

from lxml import etree

def kml_to_bbox(kml_path):
    """Extracts the bounding box from a KML file."""
    try:
        with open(kml_path, 'rb') as file:
            doc = file.read()
            try:
                
                k = kml.KML()
                try:
                    k.from_string(doc)   
                except etree.XMLSyntaxError as e:
                    print(f"XML parsing error in file {kml_path}: {e}")
                    return None
                
                features = list(k.features())
                placemarks = list(features[0].features())
                polygon = placemarks[0].geometry
                if not isinstance(polygon, Polygon):
                    polygon = shape(polygon)
                return polygon.bounds   
            except AttributeError as e:
                print(f"Error processing KML file {kml_path}: {e}")
                return None
    except FileNotFoundError:
        print(f"File not found: {kml_path}")
        return None

def generate_planet_explorer_link(bbox):
    """Generates a link to Planet Explorer based on the bounding box."""
    minx, miny, maxx, maxy = bbox
    center_lat = (miny + maxy) / 2
    center_lon = (minx + maxx) / 2
   
    zoom_level = 12
    return f"https://www.planet.com/explorer/#/center/{center_lon},{center_lat}/zoom/{zoom_level}"

kml_folder_path = "/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files"
links = []

for kml_file in os.listdir(kml_folder_path):
    kml_path = os.path.join(kml_folder_path, kml_file)
    bbox = kml_to_bbox(kml_path)
    if bbox:
        link = generate_planet_explorer_link(bbox)
        links.append(link)

 

### Generating links from explorer "https://support.planet.com/hc/en-us/articles/7506760029341-Sharing-Your-Session-in-Planet-Explorer"

In [ ]:
import os
import json
import urllib.parse
from fastkml import kml
from shapely.geometry import Polygon, shape, mapping

def parse_kml_file(kml_path):
    try:
        with open(kml_path, 'rb') as file:
            kml_content = file.read()
    except Exception as e:
        print(f"Error reading file {kml_path}: {e}")
        return None

    try:
        k = kml.KML()
        k.from_string(kml_content)
    except Exception as e:
        print(f"Error parsing KML from file {kml_path}: {e}")
        return None

    def find_polygon(features):
        for feature in features:
            if hasattr(feature, 'geometry') and feature.geometry.geom_type == 'Polygon':
                return feature.geometry
            elif hasattr(feature, 'features'):
                polygon = find_polygon(feature.features())
                if polygon:
                    return polygon
        return None

    def extract_features(element):
        if hasattr(element, 'features'):
            return element.features()
        return []

    try:
        features = []
        for doc in k.features():
            features.extend(extract_features(doc))
        return find_polygon(features)
    except Exception as e:
        print(f"Error finding polygon in file {kml_path}: {e}")
        return None

def construct_planet_explorer_url(polygon):
    if not isinstance(polygon, Polygon):
        polygon = shape(polygon)

    try:
        geojson_polygon = json.dumps(mapping(polygon))
    except Exception as e:
        print(f"Error converting polygon to GeoJSON: {e}")
        return None

    encoded_geojson = urllib.parse.quote(geojson_polygon)
    print(f"Encoded GeoJSON: {encoded_geojson}")

    centroid = polygon.centroid
    lat, lon = centroid.y, centroid.x

    # Adjust zoom level if necessary based on the polygon size
    zoom_level = 12

    url = f"https://www.planet.com/explorer/?geometry={encoded_geojson}&center={lat},{lon}&zoom={zoom_level}&show_boundaries=true"
    return url

kml_directory = "/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files"

kml_files = [f for f in os.listdir(kml_directory) if f.endswith('.kml')]

for kml_file in kml_files:
    kml_path = os.path.join(kml_directory, kml_file)
    print(f"Processing file: {kml_file}")
    polygon = parse_kml_file(kml_path)
    
    if polygon:
        url = construct_planet_explorer_url(polygon)
        if url:
            print(f"{kml_file}: {url}")
        else:
            print(f"{kml_file}: Error constructing URL")
    else:
        print(f"{kml_file}: No valid polygon found")


In [ ]:
import os
import json
import requests

# Replace with your Planet API key
API_KEY = 'PLAK37b7e85e0cec4b359fb4da4e9c62c3dd'

def create_session(kml_path):
    url = "https://api.planet.com/data/v1/graph"

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'api-key {API_KEY}'
    }

    with open(kml_path, 'r') as file:
        kml_content = file.read()

    payload = {
        "aoi": kml_content,
        "type": "kml"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        return response.json().get('links').get('self')
    else:
        print(f"Error creating session for {kml_path}: {response.text}")
        return None

kml_directory = "/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files"

kml_files = [f for f in os.listdir(kml_directory) if f.endswith('.kml')]

for kml_file in kml_files:
    kml_path = os.path.join(kml_directory, kml_file)
    print(f"Processing file: {kml_file}")
    session_link = create_session(kml_path)
    
    if session_link:
        print(f"{kml_file}: {session_link}")
    else:
        print(f"{kml_file}: Error creating session")


In [ ]:
import os

# Define the directory containing your .kml files
kml_directory = "/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Coastal_Kenya_data/kml_files"

# Get a list of all .kml files in the directory
kml_files = [f for f in os.listdir(kml_directory) if f.endswith('.kml')]

# Iterate through each .kml file, parse it, and construct the URL
for kml_file in kml_files:
    kml_path = os.path.join(kml_directory, kml_file)
    polygon = parse_kml_file(kml_path)
    
    if polygon:
        url = construct_planet_explorer_url(polygon)
        print(f"{kml_file}: {url}")
    else:
        print(f"{kml_file}: No valid polygon found")


In [ ]:
total_area_ha = komaza_df['Area_in_ha'].sum()
print(f"Total area covered: {total_area_ha} hectares")

### Latin America

In [ ]:
import pandas as pd


file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/OTHER_KML.csv'


latin= pd.read_csv(file_path)


latin.head()

In [ ]:
latin.info()

In [ ]:
total_area_ha = latin['Hectarea'].sum()
print(f"Total area covered: {total_area_ha} hectares")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

latin['Year'] = pd.to_datetime(latin['Fecha_plan']).dt.year

year_counts = latin['Year'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
year_counts.plot(kind='bar', color=plt.cm.rainbow(np.linspace(0, 1, len(year_counts))))
plt.title('Count per Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import pandas as pd


file_path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Weyerhaeuser-Eucapine N960- Project Area.csv'


WSE= pd.read_csv(file_path)

WSE["coordinates"]

In [ ]:
import re
from pyproj import Proj, transform

def latlong_to_utm(coords):
    proj_latlong = Proj(proj='latlong', datum='WGS84')
    proj_utm = Proj(proj='utm', zone=33, datum='WGS84')  
    utm_coords = [transform(proj_latlong, proj_utm, coord[1], coord[0]) for coord in coords]
    return utm_coords

def parse_polygon_coords(polygon_str):
    coord_pairs = polygon_str.split()
    coords = [tuple(map(float, coord.split(',')[:2])) for coord in coord_pairs]
    return coords

def calculate_polygon_area(coords):
    utm_coords = latlong_to_utm(coords)
    n = len(utm_coords)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += utm_coords[i][0] * utm_coords[j][1]
        area -= utm_coords[j][0] * utm_coords[i][1]
    area = abs(area) / 2.0
    return area / 10000  


WSE['Area_in_ha'] = WSE['coordinates'].apply(
    lambda x: calculate_polygon_area(parse_polygon_coords(x)) if isinstance(x, str) else None)


In [ ]:
WSE.head()

In [ ]:
total_area_ha = WSE['Area_in_ha'].sum()
print(f"Total area covered: {total_area_ha} hectares")

In [ ]:
WSE.describe()

#### concatinating datasets with common columns

In [ ]:
import pandas as pd
import glob


path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/latin_projects'  
all_files = glob.glob(path + "/*.csv")


dfs = []


for filename in all_files:

    df = pd.read_csv(filename, index_col=None, header=0)
    
    dfs.append(df)


concatenated_df_latin = pd.concat(dfs, axis=0, ignore_index=True)




In [ ]:
concatenated_df_latin.head()

In [ ]:

concatenated_df_latin['Area_in_ha'] = concatenated_df_latin['coordinates'].apply(
    lambda x: calculate_polygon_area(parse_polygon_coords(x)) if isinstance(x, str) else None)

In [ ]:
concatenated_df_latin.head()

In [ ]:
total_area_ha = concatenated_df_latin['Area_in_ha'].sum()
print(f"Total area covered: {total_area_ha} hectares")

In [ ]:



filtered_projects= concatenated_df_latin.loc[concatenated_df_latin['Area_in_ha'] < 200]
filtered_projects.info()




In [ ]:
filtered_projects.describe()

In [ ]:
total_area_ha = filtered_projects['Area_in_ha'].sum()
print(f"Total area covered: {total_area_ha} hectares")

### Asia Projects

In [ ]:

path = '/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/Asia_Project'  
all_files = glob.glob(path + "/*.csv")


dfs = []


for filename in all_files:

    df = pd.read_csv(filename, index_col=None, header=0)
    
    dfs.append(df)


concatenated_df_Asia= pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
concatenated_df_Asia.info()

In [ ]:
concatenated_df_Asia.to_csv('/home/idisc02/Forest_Project_New/Forest_Monitoring/input/Verra_Projects/TIST_Projects_without_trees/concatenated_df_Asia.csv', index=False)

In [ ]:
total_area_ha = concatenated_df_Asia['Area'].sum()
print(f"Total area covered: {total_area_ha} hectares")